In [1]:
import requests
import xmltodict
import json
import pandas as pd

#url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList'
#params ={'serviceKey' : '서비스키', 'YM' : '201201', 'NAT_CD' : '112', 'ED_CD' : 'D' }
serviceKey = 'n3K3usJdyUZFiPUsWvKo78F/Us9U9frYjiNBWtgC39z5qHHiZHWxq+4SuUg3rzOC7D6lePv0TgPmfv++C5tgzw=='
#response = requests.get(url, params=params)
#print(response.content)

In [2]:
def getTourismStateService(natCd, sexCd, ageCd, traPurpCd, portCd) :
    jsonResult = []
    result = []
    
    url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getForeignTuristStatsList'
    params = {'serviceKey' : serviceKey, 'YM' : '', 'NAT_CD': natCd, 'SEX_CD'  :  sexCd, 'AGE_CD' :  ageCd, 'TRA_PURP_CD':traPurpCd, 'PORT_CD' :portCd}
    
    monthlist = [1, 2, 3, 7]
    year = 2022
    
    for month in monthlist :
        yyyymm = "{0}{1:0>2}".format(str(year), str(month))
        try :
            params['YM'] = yyyymm
            print(params)
            response = requests.get(url,params = params)
            repResult = xmltodict.parse(response.content)
            print(repResult['response'])
            if(repResult['response']['body']['items'] == None) :
                print(f"데이터 없음. {year}년 {month}월")
                break

            natCd = repResult['response']['body']['items']['item']['natCd']
            ageCd = repResult['response']['body']['items']['item']['ageCd']
            portCd = repResult['response']['body']['items']['item']['portCd']
            traPurpCd = repResult['response']['body']['items']['item']['traPurpCd']
            sexCd = repResult['response']['body']['items']['item']['sexCd']
                
            print(f"{ageCd}{portCd}{traPurpCd}{sexCd}")
                
            jsonResult.append({'yyyymm' : yyyymm,'NAT_CD'  :  natCd,'SEX_CD'  :  sexCd, 'AGE_CD' :  ageCd, 'TRA_PURP_CD':traPurpCd, 'PORT_CD':portCd})
            result.append([yyyymm])
                
        except Exception as e :
            print(e)
                

    return jsonResult, result, natCd, sexCd, ageCd, traPurpCd, portCd
   
 

In [3]:
print("<<국내 입국한 외국인 통계 수집")

natCd = 112
sexCd = 'F'
ageCd = 30
traPurpCd = '02'
portCd = 'IA'

ed_cd = "E" #E : 방한외래관광객, D : 해외 출국

jsonResult, result, natCd, sexCd, ageCd, traPurpCd, portCd = getTourismStateService(natCd, sexCd, ageCd, traPurpCd, portCd)
print(result)

with open("%s_%s_%s_%s.json" % (sexCd, ageCd, traPurpCd, portCd),'w',encoding='utf8') as outfile :
    jsonFile = json.dumps(jsonResult, indent = 4, ensure_ascii = False)
    outfile.write(jsonFile)



    
    


<<국내 입국한 외국인 통계 수집
{'serviceKey': 'n3K3usJdyUZFiPUsWvKo78F/Us9U9frYjiNBWtgC39z5qHHiZHWxq+4SuUg3rzOC7D6lePv0TgPmfv++C5tgzw==', 'YM': '202201', 'NAT_CD': 112, 'SEX_CD': 'F', 'AGE_CD': 30, 'TRA_PURP_CD': '02', 'PORT_CD': 'IA'}
{'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': {'age': '21 - 30', 'ageCd': '30', 'natCd': '112', 'natKorNm': '중  국', 'num': '26', 'port': '인천공항', 'portCd': 'IA', 'rnum': '1', 'sex': '여성', 'sexCd': 'F', 'traPurp': '관광', 'traPurpCd': '02', 'ym': '202201'}}, 'numOfRows': '10', 'pageNo': '1', 'totalCount': '1'}}
30IA02F
{'serviceKey': 'n3K3usJdyUZFiPUsWvKo78F/Us9U9frYjiNBWtgC39z5qHHiZHWxq+4SuUg3rzOC7D6lePv0TgPmfv++C5tgzw==', 'YM': '202202', 'NAT_CD': 112, 'SEX_CD': 'F', 'AGE_CD': 30, 'TRA_PURP_CD': '02', 'PORT_CD': 'IA'}
{'header': {'resultCode': '0000', 'resultMsg': 'OK'}, 'body': {'items': {'item': {'age': '21 - 30', 'ageCd': '30', 'natCd': '112', 'natKorNm': '중  국', 'num': '22', 'port': '인천공항', 'portCd': 'IA', 'rnum': '1', 'sex': '여성

In [4]:
columns = ['입국국가', '입국연월', '성별', '연령대', '목적', '입국']
result_df = pd.DataFrame(result, columns=columns)
result_df

result_df.to_csv('%s_%s_%d_%d.csv' % (sexCd, ageCd, nStartYear, nEndYear), index = False)

ValueError: 6 columns passed, passed data had 1 columns